In [ ]:
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")#忽略不影响运行的warnings

os.chdir(r"C:\Users\lizhi\Desktop\西财\大数据与会计\2023秋\Week 6")
ERP_df = pd.read_csv("ERP发票信息1.csv",encoding="utf-8")
# 简单查看数据信息，了解数据的基本情况，对需要进行的清洗工作进行预判
ERP_df.head()

In [ ]:
"""
如果某一列的数据类型显示为object，但实际存储的是数字型数据，通常有以下两种可能原因：
1.该列包含了非数字型的数据或者缺失值（例如NaN），导致该列类型被自动解析为了object类型。
在这种情况下，需要将该列中的非数字型数据或者缺失值进行处理或替换后，再将该列数据类型转换为数字型（例如int、float）。
2.该列中存储的确实是数值型数据，但是读入数据时没有成功地解析出数据类型。
这种情况下，通常可以使用函数pd.to_numeric()对该列进行强制类型转换，以将该列数据类型转换为数字型。
"""
#发票日期、会计凭证号的格式可能需要调整；会计凭证号、产品单价中有缺失值
ERP_df. info()#查看数据属性信息

ERP_df. describe(percentiles= [0.01,0.05,0.5,0.95,0.99])


In [ ]:
# 数据清洗
## 识别和处理缺失值
print(ERP_df. isnull(). sum()) #统计数据各列缺失值的个数

In [ ]:
print(ERP_df['产品单价']. isnull())
print(ERP_df[ERP_df['产品单价']. isnull()])#输出所有产品单价缺失的行 

In [ ]:
print(ERP_df[ERP_df['会计凭证号']. isnull()]) #查看会计凭证号缺失的数据情况

In [ ]:
print(ERP_df['会计凭证号']. isnull(). sum()) #单独统计会计凭证号缺失值个数

In [ ]:
### 缺失值的处理：直接删除缺失值
"""
DataFrame.dropna(how,subset,inplace...)：根据参数的要求删除DataFrame中的缺失值
how指定根据什么规则删除缺失值，how取值为'all'或者'any'，默认为'any'
'all'表示当数据某行中（subset列的）所有值都是缺失值时，则去掉该行；
'any'表示当数据某行中（subset列的）只要有缺失值，则去掉该行
subset指定用于判断是否有缺失值的列，应为列名的列表形式（即使单列也要放在列表中），默认根据所有列进行判断
inplace指定是否替换原数据，默认inplace=False
"""
ERP_df1 = ERP_df. dropna(subset= ["会计凭证号"]) #将会计凭证号有缺失值的行去掉，将新数据赋值给ERP_df1
print(ERP_df1["会计凭证号"]. isnull(). sum())

In [ ]:
### 补全缺失值

"""
Series.fillna(value,method,inplace...)：用指定的值value或方法method来补全Series中的缺失值
value可以是一个标量，则用该标量补全所有缺失值；也可以是一个字典，则将索引标签为字典的键的缺失值补全为键对应的值；
也可以是一个Series，则根据索引标签用对应的值补全缺失值；也可以是DataFrame中的一列，同样根据索引标签用对应的值补全缺失值
method可选'backfill'(等价于'bfill')，'pad'(等价于'ffill')。
'bfill'方法用数据的下一个有效观测值来填充缺失值，'ffill'用数据的前一个有效观测值来填充缺失值；默认method=None
inplace指定是否替换原数据，默认为False
DataFrame.fillna()用法相似
"""
#### 对会计凭证号进行排序，了解其取值范围
"""
Series.sort_values(ascending,inplace...)：根据Series的值进行排序。
ascending指定是否按升序（注意不支持对中文按拼音排序），默认为True；inplace指定是否替换原数据，默认为False。
DataFrame.sort_values(by,ascending,inplace...)：根据DataFrame的by列（可以是多列）进行排序。
当by有多列时，ascending可以传入与by等长的布尔值的列表，默认按升序排序；默认不替换原数据。
Series.sort_index(level,ascending,inplace...)：根据Series的行索引进行排序。
level指定有多重索引时按哪一层索引进行排序；默认按升序排序；默认不替换原数据。
DataFrame.sort_index(axis,level,ascending,inplace...)：根据DataFrame的axis轴的索引进行排序，默认按行索引进行排序。
level指定有多重索引时按哪一层索引进行排序；默认按升序排序；默认不替换原数据。
"""
print(ERP_df['会计凭证号']. sort_values()) #可见会计凭证号取值从3001开始，因此可以考虑用0补全缺失值
print()
print(ERP_df['会计凭证号']. sort_index())#按照index number 排序
ERP_df['会计凭证号']. fillna(0,inplace=True)#用0补全缺失值
ERP_df['会计凭证号']. isnull(). sum()

缺失值的处理：数据补充（特殊值填充、统计量填充、插值法填充等）
使用同一大类的产品单价的中位数来补全缺失的产品单价

1.对数据根据产品大类进行分组
2.计算各组产品单价的中位数，并将其转换为索引与原数据一一对应的Series
3.使用此Series 来补全产品单价的缺失值

In [ ]:
"""
DataFrame.groupby(by...)：将DataFrame按照by列（可为多列名的列表）进行分组，返回一个groupby对象
可以用groupby.groups以字典形式返回各组数据的行索引
可以对groupby对象进行相应的组操作，如计算组内指定变量的统计量，对各组应用同一函数等；如groupby[column].sum()
详细内容可参考：https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html
Series.transform(func)：对Series应用func函数计算相应的值，将结果转换为新的Series，使其与原Series索引相同
详细内容可参考：https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.transform.html
"""
grouped = ERP_df. groupby('产品大类') #group 函数演示
grouped. groups

In [ ]:
print(grouped['产品单价']. median())#
print()
print(grouped['产品单价']. transform('median'))
fill = ERP_df. groupby('产品大类')['产品单价']. transform('median')#计算分组内产品单价的中位数并且生成新的series: fill并且
                                                                   #与原'产品单价'的索引相同
ERP_df['产品单价_f1'] = ERP_df['产品单价']. fillna(fill)#生成新的一列产品单价_f1， 将产品单价列中的缺失值用产品大类中位数补全后赋值过去。
ERP_df['产品单价_f1']. isnull(). sum()
ERP_df.head()

In [ ]:
#练习1
ERP_df2=ERP_df.sort_values(by=['产品大类','发票日期'],ascending =[True,True])
ERP_df2
ERP_df['产品单价_f2'] = ERP_df2. groupby('产品大类')['产品单价']. fillna(method='ffill')
ERP_df['产品单价_f2']

In [ ]:
ERP_df. info()

pd.to_datetime(arg,format...)：将对象arg转换为datetime类型，返回转换后的结果；
可自动判断转换格式（可能会误判），也可以通过format进行指定
datetime类型的Series，可以调用相应方法获得对应的日期、时间、年、月、日、时、分、秒等信息构成的Series，如
Series.dt.date：得到日期的Series
Series.dt.time：得到时间的Series
Series.dt.year：得到年的Series
Series.dt.month：得到月的Series
Series.dt.day：得到日的Series
Series.dt.hour：得到小时的Series
Series.dt.minute：得到分钟的Series
Series.dt.second：得到秒钟的Series

In [ ]:
#修正数据格式
ERP_df['发票日期'] = pd. to_datetime(ERP_df['发票日期'])
print(ERP_df['发票日期'])
ERP_df['发票日期']. dt. month

In [ ]:
#修正数据格式
ERP_df['会计凭证号'] = ERP_df['会计凭证号']. astype(int). astype(str). str. zfill(8)
#将会计凭证号转换成字符形式，
#并且运用zfill函数用0将其补满8位
ERP_df['会计凭证号']. sort_values()

识别和处理重复值
有时数据中会有重复值，有的重复值的产生是符合逻辑的，比如ERP系统中重复的订单号可能是由于同一订单分批发货产生的；有的重复值则可能是由于错误等原因造成的，比如重复的发货信息可能是由于相同业务被多次记录产生的，属于赘余信息，需要删除。
识别重复值可以使用： DataFrame.duplicated(subset,keep)：根据数据指定的列subset（可以是多列）进行重复值的判断，返回一个和数据行数相等的布尔值序列。默认 subset=None，表示根据所有列判断重复值。keep指定有重复值时如何标记，keep取值可以是'first','last',或者False；'first'是指将重复值中除了第一个以 外全部标记为True，'last'指将重复值中除了最后一个以外全部标记为True，False表示将所有的重复值都标记为True，默认keep='first'。

删除重复值可以使用： DataFrame.drop_duplicates(subset,keep,inplace...)：根据数据指定的列subset（可以是多列）删除重复值。默认根据所有列删除重复值。keep指定有重 复值时如何删除，取值可以是'first','last',或者False；'first'指保留重复值中的第一个，'last'指保留重复值中的最后一个，False指删除所有重复值，默认 keep='first'。inplace指定是否替换原数据，默认为False。 进行重复值的识别和删除之前有时需要对数据进行排序，以便对重复值进行分析和删除判断时逻辑更清晰

In [ ]:
ERP_df1 = ERP_df. sort_values(['销售订单号','发票日期']) #对数据根据销售订单号和发票日期进行排序
ERP_df1

In [ ]:
print(ERP_df1. duplicated('销售订单号',keep=False ))
ERP_df1[ERP_df1. duplicated('销售订单号',keep= False)] #获取所有销售订单号重复的数据

In [ ]:
print(ERP_df. duplicated(keep= False). sum()) #统计重复发货信息的数量（所有列都相等的数据）

In [ ]:
ERP_df. drop_duplicates(inplace=True)#删掉重复的数据

In [ ]:
ERP_df. duplicated(keep=False). sum()#检测处理后数据中重复值的情况

识别和处理异常值
在数据处理过程中，我们可能会发现有的观测值存在异常，如过高或过低，或者某些值不太合理。对于这类异常值，有时需要对其进行排除或修正，尤其是建模时，异常值可能会导致模型结论错误或预测效果不好，因此一般需要对异常值进行检测，并根据具体情况选择合适的处理方式。
异常值的识别和处理并没有一个在任何情况下均适合的标准方法，需要考虑异常产生的原因，异常的潜在影响等多方面的信息。

异常值的识别：通过描述性统计检测异常值
DataFrame.describe(percentiles...)：对DataFrame的各列求描述性统计值。 percentiles指定要展示的百分位数有哪些，应是一个由(0,1)上的数构成的列表，默认为[.25, .5, .75]

In [ ]:
### 通过描述性统计检测异常值
ERP_df[['销售收入','税','价税合计','产品单价']]. describe(percentiles= [0.01,0.99])

从各变量的统计值来看，销售收入、税、价税合计最小值都是负数，看似不太合理，但是考虑到存在退货的情况，退货会以负数表示。当然，对一个正常经营的公司来说，如果退货数量太多肯定也是不合理的，这里我们看到各变量的第1位百分位数都是正数，可以判断退货情况并不多，因此可以接受。另外还可以观察到，多个数据的均值都是远大于中位数，并且标准差与均值相比也较大，说明数据可能存在极端值

异常值的识别：利用3σ原则识别异常值
3σ原则基于正态分布的特点建立的原则。
自然界很多数据在大样本情况下会趋于正态分布，因此往往可以根据3σ原理来检测异常值。

但是要注意有的数据并不会趋于正态分布，比如金融数据很多会呈现高峰厚尾的特征，因此在应用3σ原理时需要注意这些特殊情况。

具体而言，对于样本量足够大（n>=30）且趋于服从正态分布的数据，若σ为数据标准差，μ为均值，则由正态分布的特点有以下规律：

数值分布在(μ-σ,μ+σ)中的概率为0.6826
数值分布在(μ-2σ,μ+2σ)中的概率为0.9544
数值分布在(μ-3σ,μ+3σ)中的概率为0.9974

统计中一般认为小概率事件不会发生，可以看到，数据落在离均值3倍标准差以外的区间的概率低于0.3%，属于几乎不会发生的情况。因此，如果现实中数据确实落在此区域，则可以认为可能存在异常。

In [ ]:
def three_sigma(df,columns):
    

    upper = df[columns]. mean()+ 3* df[columns]. std()
    lower = df[columns]. mean()- 3* df[columns]. std()
    outlier = (df[columns]> upper)| (df[columns]< lower)
    return outlier

In [ ]:
outlier = three_sigma(ERP_df,['销售收入','税','价税合计','产品单价'])

In [ ]:
print(outlier)
outlier.sum()#统计每一列中outlier的数量

In [ ]:
#异常值处理
#缩尾处理
"""
定义缩尾函数，对df中指定的columns进行缩尾，生成以_w结尾的缩尾变量，返回包含缩尾变量的数据
columns为需要进行缩尾的列名的列表
"""
def winsor(df,columns,lower= 0.01,upper= 0.99):

    upper_limit = df.quantile(upper) # DataFrame.quantile(q)：计算DataFrame的指定百分位数，q应为0-1上的浮点数，默认为0.5
    lower_limit = df.quantile(lower)
    df_w = df.copy()

# 浅复制有两种方式：一是，df1 = df；
# 二是，df1 = df.copy(deep=False)。即df1与df所指向的数据地址是同一个地址，其中一个变量发生变化，另一个变量对应的元素也会发生变化。
# dataframe类型数据深复制的方式为：
# df2 = df.copy(deep=True)。
# 即df2重新开辟内存空间存放df2的数据，df2的数据元素等于df的数据原始，df与df2所指向数据的地址不一样，
# 仅对应位置元素一样，故其中一个变量名中的元素发生变化，另一个不会随之发生变化

    for column in columns:
        df_w[column+ '_w'] = df_w[column]
        df_w.loc[df_w[column]> upper_limit[column],column+ '_w'] = upper_limit[column]
        df_w.loc[df_w[column]< lower_limit[column],column+ '_w'] = lower_limit[column]
    return df_w
ERP_df = winsor(ERP_df,['销售收入'])

In [ ]:
ERP_df.info

In [ ]:
ERP_df[ERP_df['销售收入_w']!= ERP_df['销售收入'] ]

In [ ]:
#练习二
CRM_df = pd. read_excel("CRM订单数据.xlsx")
CRM_df. info()
print(CRM_df. head())
print(CRM_df. duplicated(subset= '销售订单号'). sum())
print(CRM_df. isnull(). sum()) #统计数据各列缺失值的个数
CRM_df[['订单金额','订货数量','开票金额']]. describe(percentiles= [0.01,0.05,0.5,0.95,0.99])

数据集成
pd.merge使用例子
pd.merge(left,right,how,on,left_on,right_on,indicator...)：将两个数据left和right按照指定方式进行按列合并。也可以用left.merge(right...)
how指定合并的类型，how可取'left'（保留left数据的所有行和right数据中匹配上的行）,'right'（保留right数据的所有行和left数
据中匹配上的行）,'outer'（保留两个数据的所有行）,'inner'（仅保留两个数据中匹配上的行）等，默认为'inner'；
on指定两个数据合并时用于匹配的变量，可以是单列名，也可以是多列名的列表，使用on时，要求匹配变量在两个数据中都存在；
left_on和right_on用于两数据匹配变量不同名时，指定各自用于匹配的变量名；
indicator指定是否在合并数据中增加一列'_merge'用于标记合并情况，默认为False

In [ ]:
#数据集成
## pd.merge使用例子
df1 = pd. DataFrame({'id':[1,2,3,4,5],
'value1':[12,15,18,10,7]})
df2 = pd. DataFrame({'id':[0,1,3,5,7],
'value2':[2,4,6,8,10]})
df3 = pd. DataFrame({'id1':[0,1,3,5,7],
'value1':[1,3,5,7,9]})

In [ ]:
m1 = pd. merge(df1,df2,on= 'id')
m1

In [ ]:
m2 = pd. merge(df1,df2,how= 'left')
m2

In [ ]:
m3 = pd. merge(df1,df2,how= 'right')
m3

In [ ]:
m4 = pd. merge(df1,df2,how= 'outer')
m4

In [ ]:
m5 = pd. merge(df1,df3,how= 'outer',left_on= 'id',right_on= 'id1',indicator= True )
m5

pd.concat使用例子
pd.concat(objs,axis,join...)：将指定数据对象objs按轴axis进行指定方式的拼接。
objs应为用于拼接的数据组成的序列；
axis指定拼接轴，默认axis=0，表示按行拼接；axis=1则表示按列拼接。拼接时根据相同的索引进行拼接；
join指定拼接方式，可以为'inner'或'outer'，默认为'outer'。

In [ ]:
## pd.concat使用例子
c1 = pd. concat([df1,df2,df3])
c1

In [ ]:
c2 = pd. concat([df1,df2],axis= 1)
c2

In [ ]:
## ERP和CRM数据合并
merged = pd. merge(ERP_df,CRM_df,on= "销售订单号",how= "outer",indicator=True)
merged

In [ ]:
"""
pd.set_option(pat,value)：根据要求设置pandas中一些指定参数的值，可用来调整数据最大展示行数和列数
pat取'display.max_columns'时调整最大展示列数
pat取'display.max_rows'时调整最大展示行数
value则指定调整后的值，value取None时表示展示所有列或行
"""
pd. set_option('display.max_columns',None)
merged

合并情况统计
Series.value_counts(normalize,ascending,dropna...)：对Series的值出现的次数进行统计，返回以原Series的值为索引，其频数为值的Series。
normalize指定是否统计频率，默认为False，设为True时则统计频率；
ascending指定是否按频数的升序排序，默认为ascending=False；
dropna表示统计时是否剔除缺失值，默认dropna=True

In [ ]:
print(merged['_merge']. value_counts()) 
#结果显示ERP中有而CRM中没有的数据有30行，而CRM中有而ERP中没有的数据有1行
## 检查后发现ERP中的订单1007102463在CRM系统中未成功录入从而产生了30行left_only


In [ ]:
merged[merged['_merge']== 'right_only']

In [ ]:
merged[merged['_merge']== 'left_only']

In [ ]:
## 检查合并成功的数据中信息是否一致
check = merged[merged['_merge']== 'both'].copy() 
#DataFrame切片与数组切片性质相同，为避免check上的部分改动影响原数据，进行复制

In [ ]:
## 检查同一笔订单号的ERP数据中的发票金额（即价税合计）之和应该与CRM数据中的订单金额一致
check['发票金额之和'] = check. groupby('销售订单号')['价税合计']. transform('sum')

In [ ]:
check['ERP与CRM订单金额绝对差'] = abs(check['订单金额']- check['发票金额之和']) #abs(x)：取x的绝对值
check['ERP与CRM订单金额绝对差']. value_counts(). sort_index()

In [ ]:
import numpy as np
check['金额核对不一致'] = np.where(check['ERP与CRM订单金额绝对差']> 1,1,0) #因计算机数值比较非常精确，考虑容忍差异不大于1的情况
print(check['金额核对不一致'].sum())
check[check['金额核对不一致']== 1] #10条信息金额对比有较大差别，其中8条是未完成订单，2条已完成，都需要进一步检查原因

数据归一化


sklearn包的preprocessing模块提供MinMaxScaler方法可以对数据进行最值归一化，StandardScaler方法可以对数据进行均值方差归一化。初次使用sklearn包需要安装，pip install scikit-learn
用法：可以先生成scaler对象，然后调用其fit_transform()方法对数据进行相应转换，如下

scaler = MinMaxScaler() ：生成MinMaxScaler的对象scaler scaler.fit_transform(data) ：调用scaler的fit_transform方法对数据data进行最值归一化，返回变换后的数据

scaler = StandardScaler() ：生成StandardScaler的对象scaler scaler.fit_transform(data) ：调用scaler的fit_transform方法对数据data进行均值方差归一化，返回变换后的数据

注：data必须是二维的

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler #导入模块
## 对merged数据的价税合计进行最值归一化
scaler = MinMaxScaler()
merged[['价税合计_scaled','销售收入_scaled']] = scaler. fit_transform(merged[['价税合计','销售收入']])
merged[['价税合计_scaled','销售收入_scaled']]

In [ ]:
    ## 对merged数据的价税合计进行均值方差归一化
scaler = StandardScaler()
merged[['价税合计_standard','销售收入_standard']] = scaler. fit_transform(merged[['价税合计','销售收入']])
merged[['价税合计_standard','销售收入_standard']]